In [79]:
import os
from dotenv import load_dotenv
import argparse
import re
import pickle
from datetime import timedelta,datetime
import google.oauth2.credentials
import google_auth_oauthlib.flow as fl
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import boto3

In [80]:
load_dotenv()

True

In [81]:
credentials=None

In [82]:
if os.path.exists('token.pickle'):
    print('Loding Credentials from File...')
    with open('token.pickle','rb') as token:
        credentials=pickle.load(token)

Loding Credentials from File...


In [83]:
if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            print('Refereshing access token...')
            credentials.refresh(Request())
        else:
            print('Fetching New Tokens...')
            flow=fl.InstalledAppFlow.from_client_secrets_file('client_secret.json',scopes=['https://www.googleapis.com/auth/youtube.readonly'])
            flow.run_local_server(port=8080, prompt='consent',authorization_prompt_message='')
            credentials=flow.credentials
        
            with open('token.pickle','wb') as f:
                print('Saving credentials for future use...')
                pickle.dump(credentials,f)
    
        #credentials=flow.credentials
    #print(credentials.to_json())


Refereshing access token...


In [84]:
youtube=build("youtube","v3",credentials=credentials)

In [85]:
#Pulling all the playlists for my channel

channel_id='UC-B_ggoiilgcbihS-GYBnIA'

playlist_list=youtube.playlists().list(
    part='contentDetails,id,snippet',
    channelId=channel_id,
    maxResults=25
    )
playlist_lists=playlist_list.execute()

lst=[]
for item in playlist_lists['items']:
    lst.append({'playlist_id':item['id'],'playlist_name':item['snippet']['title'],'no_of_videos':item['contentDetails']['itemCount']})

df=pd.DataFrame(lst)
df.head()

Pulling the necessary data for analysis purpose

In [87]:
playlist_id='PL_wgt95pDQnwywGe6nd7C4-Np1IIFKCgD'

vid_ids=[]
nextpagetoken=None

while True:
    playlist_vid=youtube.playlistItems().list(
        part='contentDetails,snippet',
        playlistId=playlist_id,
        maxResults=50,
        pageToken=nextpagetoken
        )
    playlist_vids=playlist_vid.execute()

    for item in playlist_vids['items']:
        vid_ids.append({
                            'video_id':item['contentDetails']['videoId'],
                            'publisheddate':item.get('contentDetails',{}).get('videoPublishedAt',{}),
                            'video_title':item['snippet']['title'],
                            'videoOwnerChannelTitle':item.get('snippet',{}).get('videoOwnerChannelTitle',{}),
                            'thumbnail':item.get('snippet',{}).get('thumbnails',{}).get('medium',{}).get('url',{})
                        })
    
    nextpagetoken=playlist_vids.get('nextPageToken')
    
    if not nextpagetoken:
        break
    

df2=pd.DataFrame(vid_ids)

In [88]:
df2.shape

(447, 5)

In [89]:
chunk_size = 50
videos = []
music_category = []

for i in range(0, len(vid_ids), chunk_size):
    chunk = vid_ids[i:i + chunk_size]
    video_ids = ','.join(item['video_id'] for item in chunk)

    vid_request = youtube.videos().list(
        part='statistics,contentDetails,snippet,topicDetails',
        id=video_ids
    )
    vid_response = vid_request.execute()

    for item in vid_response['items']:
        vid_views = item['statistics']['viewCount']
        vid_like_count = item.get('statistics',{}).get('likeCount',0)
        vid_comment_count = item.get('statistics').get('commentCount',0)
        video_id = item['id']
        yt_link = f'https://youtu.be/{video_id}'
        category_id = item['snippet']['categoryId']
        categories = item.get('topicDetails', {}).get('topicCategories', [])
        category_value = list(set(category.split('/')[-1] for category in categories))
        music_category.append({'video_id': video_id, 'category_value': category_value})

        videos.append({
            'views': int(vid_views),
            'url': yt_link,
            'video_id': video_id,
            'video_likes': int(vid_like_count),
            'video_comments': int(vid_comment_count),
            'video_category_id': int(category_id)
        })

# Convert the list of video details to a DataFrame
df3 = pd.DataFrame(videos)

# Merge the two DataFrames on 'video_id'
df4 = df2.merge(df3, on='video_id')

# Convert the list of music categories to a DataFrame
df5 = pd.DataFrame(music_category)
df_new=df4.merge(df5,on='video_id')

In [90]:
df_new.shape

(440, 11)

In [91]:
df_new['category_value']=df_new['category_value'].apply(','.join)
df_new['publisheddate']= pd.to_datetime(df_new['publisheddate'])

In [ ]:
#df_new['tags']=df_new['videoOwnerChannelTitle']+df_new['category_value']
#df_new['tags']=df_new['tags'].apply(lambda x:x.lower())

Performing some EDA

In [ ]:
df_sorted = df_new.sort_values(by='views', ascending=False)

# Plot a bar chart for the top 10 values
plt.figure(figsize=(10, 6))
plt.barh(df_sorted['video_title'].head(10), df_sorted['views'].head(10), color='skyblue')
plt.xlabel('Total No of Views')
plt.ylabel('Song Name')
plt.title('Top 10 Songs based on views')
plt.show()

In [ ]:
df_sorted2 = df_new.sort_values(by='video_likes', ascending=False)

# Plot a bar chart for the top 10 values
plt.figure(figsize=(10, 6))
plt.barh(df_sorted2['video_title'].head(10), df_sorted2['video_likes'].head(10), color='skyblue')
plt.xlabel('Total No of Likes')
plt.ylabel('Sonog Name')
plt.title('Top 10 Songs based on likes')
plt.show()

In [ ]:
df_sorted3 = df_new.sort_values(by='video_comments', ascending=False)

# Plot a bar chart for the top 10 values
plt.figure(figsize=(10, 6))
plt.barh(df_sorted3['video_title'].head(10), df_sorted3['video_comments'].head(10), color='skyblue')
plt.xlabel('Total No of Comments')
plt.ylabel('Song Name')
plt.title('Top 10 Songs based on Comments')
plt.show()

In [ ]:
df_new['publisheddate'] = df_new['publisheddate'].dt.strftime('%Y-%m-%d')
df_new['year'] = pd.to_datetime(df_new['publisheddate']).dt.year
grouped_df=df_new.groupby(df_new['year']).size()

plt.figure(figsize=(6, 6))
plt.pie(grouped_df, labels=grouped_df.index, autopct='%1.1f%%', startangle=140)
plt.title('Records by Year')
plt.show()

In [ ]:
columns_to_normalize=['views','video_likes','video_comments']
data_to_normalize = df_new[columns_to_normalize]

In [ ]:
scaler=MinMaxScaler()

In [ ]:
scaler.fit(data_to_normalize)

In [ ]:
df_new[columns_to_normalize]=scaler.transform(data_to_normalize)

In [ ]:
'''I created this rating formula where I am giving 50% weightage to views, 
30% weightage to likes and 20% weightage to comments to see top 50 songs in my playlist'''
df_new['Rating']=df_new['views']*0.5+df_new['video_likes']*0.3+df_new['video_comments']*0.2

In [ ]:
popular_songs=df_new.sort_values(by='Rating',ascending=False).head(50)

In [ ]:
popular_songs.to_csv('Popular_Songs.csv')
ct=datetime.now().strftime('%Y%m%d_%H%M%S')

s3=boto3.client('s3')



bucket_name='bucket1swap'
s3_folder='Snowflake_pipe/'
file_name='Playlist_Songs_{}.csv'.format(ct)
df_new.to_csv(file_name,index=False)
s3.upload_file(file_name,bucket_name,s3_folder+file_name)